In [1]:
!pip install tweepy

     |████████████████████████████████| 69 kB 565 kB/s            
     |████████████████████████████████| 151 kB 1.2 MB/s            
     |████████████████████████████████| 63 kB 845 kB/s             
  Attempting uninstall: requests
    Found existing installation: requests 2.26.0
    Uninstalling requests-2.26.0:
      Successfully uninstalled requests-2.26.0
  Attempting uninstall: oauthlib
    Found existing installation: oauthlib 3.1.1
    Uninstalling oauthlib-3.1.1:
      Successfully uninstalled oauthlib-3.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
explainable-ai-sdk 1.3.2 requires xai-image-widget, which is not installed.
beatrix-jupyterlab 3.1.6 requires google-cloud-bigquery-storage, which is not installed.
spacy 3.2.1 requires typing-extensions<4.0.0.0,>=3.7.4; python_version < "3.8", but you have typing-extensions 4.0.1 which is incompat

In [2]:
import tweepy
from tweepy import OAuthHandler
import json
import datetime as dt
import pytz
import time
import os
import pandas as pd
import sys
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
access_token = user_secrets.get_secret("access_token")
access_token_secret = user_secrets.get_secret("access_token_secret")
api_key = user_secrets.get_secret("api_key")
api_key_secret = user_secrets.get_secret("api_key_secret")
#Setup Kaggle OS ENV to upload data
os.environ["KAGGLE_USERNAME"] = user_secrets.get_secret("kaggle_username")
os.environ["KAGGLE_KEY"] = user_secrets.get_secret("kaggle_key")

#sendgrid
sendgrid_api_key = user_secrets.get_secret("sendgrid_api_key")

In [4]:
def load_api():
    ''' Function that loads the twitter API after authorizing the user. '''
    #Setup authentication for Tweepy
    auth = tweepy.OAuthHandler(api_key,api_key_secret)
    auth.set_access_token(access_token,access_token_secret)
    # load the twitter API via tweepy
    return tweepy.API(auth)

In [5]:
def tweet_text_refine(text):
    text.replace('\n\n', '\\n').replace('\n', '\\n')
    return text

In [6]:
def tweet_search(api, query, max_tweets, max_id, since_id, wordle_id):
    ''' Function that takes in a search string 'query', the maximum
        number of tweets 'max_tweets', and the minimum (i.e., starting)
        tweet id. It returns a list of tweepy.models.Status objects. '''

    searched_tweets = []
    while len(searched_tweets) < max_tweets:
        remaining_tweets = max_tweets - len(searched_tweets)
        try:
            new_tweets = api.search_tweets(q=query, count=remaining_tweets,
                                    since_id=str(since_id),
				                    max_id=str(max_id-1))
#                                    geocode=geocode)
            print('found',len(new_tweets),'tweets')
            if not new_tweets:
                print('no tweets found')
                break
            for tweet in new_tweets:
                searched_tweets.append([wordle_id,tweet.id,tweet.created_at,tweet_text_refine(tweet.text),tweet.source,tweet.user.id,tweet.user.name,tweet.user.screen_name,tweet.user.location,tweet.truncated])
            max_id = new_tweets[-1].id
        except tweepy.errors.TweepyException:
            print('exception raised, waiting 15 minutes')
            print('(until:', dt.datetime.now()+dt.timedelta(minutes=15), ')')
            time.sleep(15*60)
            break # stop the loop
    return searched_tweets, max_id

In [7]:
def get_tweet_id(api, date='', days_ago=9, query='a'):
    ''' Function that gets the ID of a tweet. This ID can then be
        used as a 'starting point' from which to search. The query is
        required and has been set to a commonly used word by default.
        The variable 'days_ago' has been initialized to the maximum
        amount we are able to search back in time (9).'''

    if date:
        # return an ID from the start of the given day
        td = date + dt.timedelta(days=1)
        tweet_date = '{0}-{1:0>2}-{2:0>2}'.format(td.year, td.month, td.day)
        tweet = api.search_tweets(tweet_dateq=query, count=1, until=tweet_date)
    else:
        # return an ID from __ days ago
        td = dt.datetime.now() - dt.timedelta(days=days_ago)
        tweet_date = '{0}-{1:0>2}-{2:0>2}'.format(td.year, td.month, td.day)
        # get list of up to 10 tweets
        tweet = api.search_tweets(q=query, count=10, until=tweet_date)
        print('search limit (start/stop):',tweet[0].created_at)
        # return the id of the first tweet in the list
        return tweet[0]

In [8]:
def write_tweets(tweets, filename):
    ''' Function that appends tweets to a file. '''

    # Make data frame of above data
    df = pd.DataFrame(tweets)
 
    # append data frame to CSV file
    df.to_csv(filename, mode='a', index=False, header=False,na_rep='Unknown')
    return len(df)

This will help us get the current wordle id. Since it would be difficult to get 

In [9]:
def get_wordle_id():
    wordle_start = pytz.timezone("US/Pacific").localize(dt.datetime(2021, 6, 19))
    now = pytz.utc.localize(dt.datetime.now()).astimezone(pytz.timezone("US/Pacific"))
    wordle_id = (now-wordle_start).days
    return wordle_id
    
    

In [10]:
get_wordle_id()

264

In [11]:
def send_mail(content):
    mail_from = 'sahilvora0409@gmail.com'
    mail_to = 'sahilvora0409@gmail.com'
    msg = MIMEMultipart()
    msg['From'] = mail_from
    msg['To'] = mail_to
    msg['Subject'] = 'Started script for wordle_id' + str(content['wordle_id'])
    mail_body = f"""
    Hey,

    The script started running for wordle id {content['wordle_id']}
    Start_date : {content['start_date']}
    End_date : {content['end_date']}
    Script started at: {dt.datetime.now()}
    Regards,\nSahil

    """
    msg.attach(MIMEText(mail_body))

    try:
        server = smtplib.SMTP_SSL('smtp.sendgrid.net', 465)
        server.ehlo()
        server.login('apikey', sendgrid_api_key)
        server.sendmail(mail_from, mail_to, msg.as_string())
        server.close()
        print("mail sent")
    except:
        print("issue")

Variables for the script

In [12]:
wordle_id = get_wordle_id()
search_phrase = 'Wordle '+ str(wordle_id)
max_count = 20000                          # max count for a wordle id
max_tweets = 100                           # number of tweets per search (will be
                                           # iterated over) - maximum is 100
min_days_old, max_days_old = 1, 2          # search limits e.g., from 7 to 8
                                           # gives current weekday from last week,
                                           # min_days_old=0 will search from right now

cols = ['WordleID','ID','Created_At','Text','Source','UserID','Username','User_ScreenName','Location','Truncated']


In [13]:
print('Search phrase =', search_phrase)

''' other variables '''
if not os.path.exists("updated"):
    os.mkdir("updated")

name = 'WordleMegaData'
csv_file_root = 'updated' + '/'  + name
os.makedirs(os.path.dirname(csv_file_root), exist_ok=True)
read_IDs = False

# open a file in which to store the tweets
if max_days_old - min_days_old == 1:
    d = dt.datetime.now() - dt.timedelta(days=min_days_old)
    print('Checking date',d)
    day = '{0}-{1:0>2}-{2:0>2}'.format(d.year, d.month, d.day)
else:
    d1 = dt.datetime.now() - dt.timedelta(days=max_days_old-1)
    d2 = dt.datetime.now() - dt.timedelta(days=min_days_old)
    day = '{0}-{1:0>2}-{2:0>2}_to_{3}-{4:0>2}-{5:0>2}'.format(
        d1.year, d1.month, d1.day, d2.year, d2.month, d2.day)
csv_file = csv_file_root + '.csv'
if os.path.isfile(csv_file):
    print('Appending tweets to file named: ',csv_file)
    read_IDs = True

# authorize and load the twitter API
api = load_api()
# set the 'starting point' ID for tweet collection
if read_IDs:
    # open the csv file and get the latest tweet ID
    df = pd.read_csv(csv_file)
    if not (df.loc[df['WordleID'] == wordle_id]).empty:
        max_id = df.loc[df['WordleID'] == wordle_id].iloc[-1]['ID']
        print('Searching from the bottom ID in file')
    else:
        if min_days_old == 0:
            max_id = -1
        else:
            tweet = get_tweet_id(api, days_ago=(min_days_old-1))
            max_id = tweet.id
            s_date = tweet.created_at
else:
    df = pd.DataFrame(columns=cols)
    df.to_csv(csv_file,index=False)
    # get the ID of a tweet that is min_days_old
    if min_days_old == 0:
        max_id = -1
    else:
        tweet = get_tweet_id(api, days_ago=(min_days_old-1))
        max_id = tweet.id
        s_date = tweet.created_at
# set the smallest ID to search for
tweet = get_tweet_id(api, days_ago=(max_days_old-1))
since_id = tweet.id
e_date = tweet.created_at
print('max id (starting point) =', max_id)
print('since id (ending point) =', since_id)

content = {}
content['wordle_id'] = wordle_id
content['start_date'] = s_date
content['end_date'] = e_date

send_mail(content)


''' tweet gathering loop  '''
count, exitcount = 0, 0
rows = len(df.loc[df['WordleID'] == wordle_id])
while rows < max_count:
    count += 1
    print('count =',count)
    # collect tweets and update max_id

    tweets, max_id = tweet_search(api, search_phrase, max_tweets,
                                max_id=max_id, since_id=since_id,wordle_id = wordle_id)
    # write tweets to file in JSON format
    if tweets:
        rows += write_tweets(tweets, csv_file)
        exitcount = 0
    else:
        exitcount += 1
        if exitcount == 3:
            sys.exit('Maximum number of empty tweet strings reached - exiting')


Search phrase = Wordle 264
Checking date 2022-03-10 02:40:14.495156
search limit (start/stop): 2022-03-10 23:59:59+00:00
search limit (start/stop): 2022-03-09 23:59:59+00:00
max id (starting point) = 1502071804512415745
since id (ending point) = 1501709416650842113
mail sent
count = 1
found 100 tweets
count = 2
found 100 tweets
count = 3
found 100 tweets
count = 4
found 100 tweets
count = 5
found 100 tweets
count = 6
found 100 tweets
count = 7
found 100 tweets
count = 8
found 100 tweets
count = 9
found 100 tweets
count = 10
found 100 tweets
count = 11
found 100 tweets
count = 12
found 100 tweets
count = 13
found 100 tweets
count = 14
found 100 tweets
count = 15
found 100 tweets
count = 16
found 100 tweets
count = 17
found 100 tweets
count = 18
found 100 tweets
count = 19
found 100 tweets
count = 20
found 100 tweets
count = 21
found 100 tweets
count = 22
found 100 tweets
count = 23
found 100 tweets
count = 24
found 100 tweets
count = 25
found 100 tweets
count = 26
found 100 tweets
count

In [14]:
df = pd.read_csv("./updated/WordleMegaData.csv")
df

,WordleID,ID,Created_At,Text,Source,UserID,Username,User_ScreenName,Location,Truncated
0,264,1502071797855965186,2022-03-10 23:59:58+00:00,Wordle 264 3/6\n\n🟨⬛⬛⬛⬛\n🟩🟩⬛🟨⬛\n🟩🟩🟩🟩🟩,Twitter for iPhone,14614922,J. Phillipe Nelson,cibertek,"Atlanta, GA",False
1,264,1502071797017104404,2022-03-10 23:59:58+00:00,Wordle 264 3/6\n\n🟨⬛🟨🟨🟨\n🟩⬛🟨🟩🟩\n🟩🟩🟩🟩🟩,Twitter for Android,3061806541,Steve LVI,100prcentGringo,"Overland Park, KS",False
2,264,1502071795997945862,2022-03-10 23:59:57+00:00,RT @dom_tor: Wordle 264 3/6 ⁦@melguarino⁩\n\n⬛...,Twitter for iPhone,243443999,Mel Guarino,melguarino,"Toronto, Ontario",False
3,264,1502071781317873670,2022-03-10 23:59:54+00:00,Done with a minute to spare \nWordle 264 6/6\n...,Twitter for iPhone,453551807,Jann Arpino,Jalepino82,Edinburgh,False
4,264,1502071777526173700,2022-03-10 23:59:53+00:00,Wordle 264 4/6\n\n🟨⬛⬛🟨⬛\n⬛🟩🟨🟨🟨\n🟨🟩🟨⬛🟨\n🟩🟩🟩🟩🟩\n...,Twitter for iPhone,559942862,Dautri 💫,_dautcom,NaN,False
...,...,...,...,...,...,...,...,...,...,...
19995,264,1501986257391529989,2022-03-10 18:20:03+00:00,Wordle 264 3/6\n\n⬜⬜⬜🟨🟨\n🟨🟨🟨🟨🟨\n🟩🟩🟩🟩🟩,Twitter for Android,615386085,Sarah McArdle,SarahLM11,Canada,False
19996,264,1501986253608263680,2022-03-10 18:20:03+00:00,Wordle 264 4/6\n\n⬛⬛⬛⬛⬛\n🟨⬛🟨🟨🟨\n🟨🟨🟨🟨⬛\n🟩🟩🟩🟩🟩\n...,Twitter for iPhone,530835203,✨xTKOxL3GACYx ✨,xTheLostLegendx,The Dreadnaught,False
19997,264,1501986250894495745,2022-03-10 18:20:02+00:00,Wordle 264 2/6\n\n🟩🟩⬜🟨🟨\n🟩🟩🟩🟩🟩,Twitter for Android,395838658,Ori'aku,superkweeen,NaN,False
19998,264,1501986248512049173,2022-03-10 18:20:01+00:00,Wordle 264 4/6\n\n⬛⬛⬛🟨🟨\n🟩🟨🟩⬛⬛\n🟩🟩🟩⬛🟩\n🟩🟩🟩🟩🟩\n...,Twitter for iPhone,403138409,Ted Cadigan,TedCadigan,"Greater Vancouver, British Columbia",False


In [15]:
previous_df = pd.read_csv("../input/wordletweets/WordleMegaData.csv")
previous_df

,WordleID,ID,Created_At,Text,Source,UserID,Username,User_ScreenName,Location,Truncated
0,254,1498447921448034305,2022-02-28 23:59:58+00:00,Wordle 254 3/6\n\n🟨🟨⬛⬛⬛\n🟨⬛🟨⬛🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,36816757,Leslie Brown,live_laugh_pray,Haida Gwaii,False
1,254,1498447918184996864,2022-02-28 23:59:58+00:00,Wordle 254 4/6\n\n⬛⬛⬛⬛🟩\n⬛⬛⬛⬛🟩\n⬛🟩⬛⬛🟩\n🟩🟩🟩🟩🟩\n...,Twitter for iPhone,482591014,Dylan Sobo,DaBolt727,"Largo, FL / Bradenton|FGCU",False
2,254,1498447910173921282,2022-02-28 23:59:56+00:00,Wordle 254 3/6\n\n⬛⬛⬛⬛🟩\n⬛⬛🟩⬛🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,1397624255626584074,🖤,wengojos,23 • she/her • 🇵🇭🇺🇸,False
3,254,1498447901797801989,2022-02-28 23:59:54+00:00,Wordle 254 3/6\n\n⬛⬛⬛🟨⬛\n⬛🟨⬛⬛🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,1255905442548473856,Dustin Waters,DustinWaters12,NaN,False
4,254,1498447896911527938,2022-02-28 23:59:53+00:00,Wordle 254 3/6\n\n⬛🟨⬛⬛⬛\n🟩🟩🟩⬛🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,252893719,R. Colin,rollin_fatty,"Richmond, VA",False
...,...,...,...,...,...,...,...,...,...,...
199995,263,1501620811412738051,2022-03-09 18:07:54+00:00,Wordle 263 4/6\n\n⬜⬜⬜🟨⬜\n🟨⬜⬜🟨🟨\n🟨🟨⬜🟨⬜\n🟩🟩🟩🟩🟩,Twitter for Android,1460828096731533320,Storm Mith,storm_mith,NaN,False
199996,263,1501620810942976002,2022-03-09 18:07:54+00:00,Wordle 263 4/6\n\n🟨⬛⬛⬛🟨\n🟨🟨🟨⬛🟨\n🟨🟩⬛🟩🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,1429177306648612873,smugish,smugish1,NaN,False
199997,263,1501620810762575876,2022-03-09 18:07:54+00:00,Wordle 263 3/6*\n\n⬛🟨🟨🟨⬛\n🟨🟩⬛🟩🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,243524667,Perdomo,AntP_FS,NaN,False
199998,263,1501620810708049923,2022-03-09 18:07:54+00:00,Wordle 263 4/6\n\n🟨⬛⬛⬛⬛\n🟨🟩⬛⬛🟩\n⬛🟩⬛🟩🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,1150115268603068426,rechelle,rechelleku,ur moms house,False


In [16]:
previous_df.groupby("WordleID")["WordleID"].count()

WordleID
254    20000
255    20000
256    20000
257    20000
258    20000
259    20000
260    20000
261    20000
262    20000
263    20000
Name: WordleID, dtype: int64

In [17]:
if len(previous_df.loc[previous_df['WordleID'] == wordle_id]) == 0:
    print('ok to add')
    new_df = (pd.concat([previous_df,df])
      .drop_duplicates(subset=["WordleID","ID"])
      .sort_values(by=["WordleID","ID"],ascending = [True, False])
      .reset_index(drop=True))
    new_df.to_csv("./updated/WordleMegaData.csv", index=False)
    !kaggle datasets metadata -p updated vora1011/wordletweets
    !kaggle datasets version -m "updated wordle tweets" -p updated
    
    

ok to add
Downloaded metadata to updated/dataset-metadata.json
Starting upload for file WordleMegaData.csv
100%|██████████████████████████████████████| 48.9M/48.9M [00:04<00:00, 11.4MB/s]
Upload successful: WordleMegaData.csv (49MB)
Dataset version is being created. Please check progress at https://www.kaggle.com/vora1011/wordletweets


In [18]:
new_df

,WordleID,ID,Created_At,Text,Source,UserID,Username,User_ScreenName,Location,Truncated
0,254,1498447921448034305,2022-02-28 23:59:58+00:00,Wordle 254 3/6\n\n🟨🟨⬛⬛⬛\n🟨⬛🟨⬛🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,36816757,Leslie Brown,live_laugh_pray,Haida Gwaii,False
1,254,1498447918184996864,2022-02-28 23:59:58+00:00,Wordle 254 4/6\n\n⬛⬛⬛⬛🟩\n⬛⬛⬛⬛🟩\n⬛🟩⬛⬛🟩\n🟩🟩🟩🟩🟩\n...,Twitter for iPhone,482591014,Dylan Sobo,DaBolt727,"Largo, FL / Bradenton|FGCU",False
2,254,1498447910173921282,2022-02-28 23:59:56+00:00,Wordle 254 3/6\n\n⬛⬛⬛⬛🟩\n⬛⬛🟩⬛🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,1397624255626584074,🖤,wengojos,23 • she/her • 🇵🇭🇺🇸,False
3,254,1498447901797801989,2022-02-28 23:59:54+00:00,Wordle 254 3/6\n\n⬛⬛⬛🟨⬛\n⬛🟨⬛⬛🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,1255905442548473856,Dustin Waters,DustinWaters12,NaN,False
4,254,1498447896911527938,2022-02-28 23:59:53+00:00,Wordle 254 3/6\n\n⬛🟨⬛⬛⬛\n🟩🟩🟩⬛🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,252893719,R. Colin,rollin_fatty,"Richmond, VA",False
...,...,...,...,...,...,...,...,...,...,...
219995,264,1501986257391529989,2022-03-10 18:20:03+00:00,Wordle 264 3/6\n\n⬜⬜⬜🟨🟨\n🟨🟨🟨🟨🟨\n🟩🟩🟩🟩🟩,Twitter for Android,615386085,Sarah McArdle,SarahLM11,Canada,False
219996,264,1501986253608263680,2022-03-10 18:20:03+00:00,Wordle 264 4/6\n\n⬛⬛⬛⬛⬛\n🟨⬛🟨🟨🟨\n🟨🟨🟨🟨⬛\n🟩🟩🟩🟩🟩\n...,Twitter for iPhone,530835203,✨xTKOxL3GACYx ✨,xTheLostLegendx,The Dreadnaught,False
219997,264,1501986250894495745,2022-03-10 18:20:02+00:00,Wordle 264 2/6\n\n🟩🟩⬜🟨🟨\n🟩🟩🟩🟩🟩,Twitter for Android,395838658,Ori'aku,superkweeen,NaN,False
219998,264,1501986248512049173,2022-03-10 18:20:01+00:00,Wordle 264 4/6\n\n⬛⬛⬛🟨🟨\n🟩🟨🟩⬛⬛\n🟩🟩🟩⬛🟩\n🟩🟩🟩🟩🟩\n...,Twitter for iPhone,403138409,Ted Cadigan,TedCadigan,"Greater Vancouver, British Columbia",False


In [19]:
new_df.groupby("WordleID")["WordleID"].count()

WordleID
254    20000
255    20000
256    20000
257    20000
258    20000
259    20000
260    20000
261    20000
262    20000
263    20000
264    20000
Name: WordleID, dtype: int64